In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **import libraries**

In [ ]:
import os
import json
import shutil
from pathlib import Path
import cv2
from tqdm import tqdm

In [ ]:
# Translation mapping from Turkish to English
replacement_map  = {
    "saglam": "Healthy",
    "g\u00f6m\u00fcl\u00fc": "Impacted",
    "\u00e7\u00fcr\u00fck": "Caries",
    "k\u00fcretaj": "Deep Caries",
    "kanal": "Periapical Lesion",
    "çekim": "Tooth extraction",
    "lezyon": "Lesion",
    "kırık": "Fracture",
}
# Supported image extensions (customize if needed)
IMAGE_EXTENSIONS = [".png", ".jpg", ".jpeg"]
# Only remove shapes whose middle label part matches exactly
labels_to_remove = {"Healthy", "Tooth extraction", "Lesion", "Fracture"}
# Define your label map here
label2id = {
    "Impacted": 0,
    "Caries": 1,
    "Deep Caries": 2,
    "Periapical Lesion": 3
}

In [ ]:
def Translation(folder_path, save_to_same_files=True, new_folder=None):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".json"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                changed = False
                for shape in data.get("shapes", []):
                    parts = shape.get("label", "").split("-")
                    if len(parts) == 3:
                        old_label = parts[1].strip()
                        new_label = replacement_map.get(old_label, old_label)
                        if old_label != new_label:
                            parts[1] = new_label
                            shape["label"] = "-".join(parts)
                            changed = True

                if changed:
                    if save_to_same_files:
                        save_path = file_path
                    else:
                        assert new_folder is not None, "You must specify new_folder if save_to_same_files=False"
                        rel_path = os.path.relpath(root, folder_path)
                        target_dir = os.path.join(new_folder, rel_path)
                        os.makedirs(target_dir, exist_ok=True)
                        save_path = os.path.join(target_dir, filename)

                    with open(save_path, "w", encoding="utf-8") as f:
                        json.dump(data, f, ensure_ascii=False, indent=4)
                    print(f"Updated: {save_path}")

#-----------------------------------------------------------------------------------------------------------------
def extract_condition(label_string):
    parts = label_string.split("-")
    if len(parts) == 3:
        return parts[1]  # "6-Impacted-28" → "Impacted"
    return None  # For labels not matching the pattern
#-----------------------------------------------------------------------------------------------------------------
def clean_json_file(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    original_shapes = data.get("shapes", [])
    cleaned_shapes = []
    removed_count = 0

    for shape in original_shapes:
        condition = extract_condition(shape["label"])
        if condition not in labels_to_remove:
            cleaned_shapes.append(shape)
        else:
            removed_count += 1

    if removed_count > 0:
        data["shapes"] = cleaned_shapes
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4)
        print(f"Cleaned {json_path}: removed {removed_count} shapes")
#-----------------------------------------------------------------------------------------------------------------
def delete_json_and_image_if_shapes_empty(json_path, image_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    if not data.get("shapes"):  # shapes list is empty
        # Delete JSON
        os.remove(json_path)
        print(f"Deleted JSON: {json_path}")

        # Get image file name
        base_name = os.path.splitext(os.path.basename(json_path))[0]
        for ext in IMAGE_EXTENSIONS:
            image_path = os.path.join(image_dir, base_name + ext)
            if os.path.exists(image_path):
                os.remove(image_path)
                print(f"Deleted Image: {image_path}")
                break
#-----------------------------------------------------------------------------------------------------------------
def clean_all_jsons(label_dir, image_dir):
    for file in os.listdir(label_dir):
        if file.endswith(".json"):
            json_path = os.path.join(label_dir, file)
            clean_json_file(json_path)
            delete_json_and_image_if_shapes_empty(json_path, image_dir)
#-----------------------------------------------------------------------------------------------------------------
def fix_impacted_label_ids(label_dir):
    for filename in os.listdir(label_dir):
        if not filename.endswith(".json"):
            continue

        file_path = os.path.join(label_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        changed = False
        for shape in data.get("shapes", []):
            label = shape.get("label", "")
            parts = label.split("-")
            if len(parts) == 3 and parts[0] == "6" and parts[1] == "Impacted":
                parts[0] = "0"
                shape["label"] = "-".join(parts)
                changed = True

        if changed:
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            print(f"✅ Fixed: {filename}")
#-----------------------------------------------------------------------------------------------------------------
def labelme_to_coco(label_dir, image_dir, output_json):
    coco = {
        "images": [],
        "annotations": [],
        "categories": [{"id": v, "name": k} for k, v in label2id.items()]
    }

    image_id = 0
    ann_id = 0

    for filename in tqdm(os.listdir(label_dir)):
        if not filename.endswith(".json"):
            continue

        json_path = os.path.join(label_dir, filename)
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Get image name and path
        image_name = data.get("imagePath", filename.replace(".json", ".png"))
        image_path = os.path.join(image_dir, image_name)

        if not os.path.exists(image_path):
            print(f"⚠️ Skipping: image not found for {filename}")
            continue

        # Get image size
        img = cv2.imread(image_path)
        if img is None:
            print(f"⚠️ Cannot read image: {image_name}")
            continue
        h, w = img.shape[:2]

        # Add image info
        coco["images"].append({
            "id": image_id,
            "file_name": image_name,
            "width": w,
            "height": h
        })

        # Process each shape
        for shape in data.get("shapes", []):
            label_str = shape["label"].strip()
            parts = label_str.split("-")

            if len(parts) != 3:
                continue  # skip malformed

            disease_name = parts[1]
            if disease_name not in label2id:
                continue  # skip unknown labels

            points = shape["points"]
            x_coords = [p[0] for p in points]
            y_coords = [p[1] for p in points]
            x_min, y_min = min(x_coords), min(y_coords)
            x_max, y_max = max(x_coords), max(y_coords)

            bbox = [x_min, y_min, x_max - x_min, y_max - y_min]
            area = bbox[2] * bbox[3]

            coco["annotations"].append({
                "id": ann_id,
                "image_id": image_id,
                "category_id": label2id[disease_name],
                "bbox": bbox,
                "area": area,
                "iscrowd": 0
            })
            ann_id += 1

        image_id += 1

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(coco, f, indent=4)
    print(f"\n✅ COCO-style annotation saved to: {output_json}")

In [ ]:
LABEL_DIR = "/content/drive/MyDrive/dentex_test/label"
IMAGE_DIR ="/content/drive/MyDrive/dentex_test/input"
OUTPUT_PATH = "/content/drive/MyDrive/dentex_test/Test_coco.json"

In [ ]:
Translation(LABEL_DIR)

Updated: /content/drive/MyDrive/dentex_test/label/test_137.json
Updated: /content/drive/MyDrive/dentex_test/label/test_203.json
Updated: /content/drive/MyDrive/dentex_test/label/test_167.json
Updated: /content/drive/MyDrive/dentex_test/label/test_11.json
Updated: /content/drive/MyDrive/dentex_test/label/test_28.json
Updated: /content/drive/MyDrive/dentex_test/label/test_88.json
Updated: /content/drive/MyDrive/dentex_test/label/test_109.json
Updated: /content/drive/MyDrive/dentex_test/label/test_223.json
Updated: /content/drive/MyDrive/dentex_test/label/test_80.json
Updated: /content/drive/MyDrive/dentex_test/label/test_151.json
Updated: /content/drive/MyDrive/dentex_test/label/test_172.json
Updated: /content/drive/MyDrive/dentex_test/label/test_157.json
Updated: /content/drive/MyDrive/dentex_test/label/test_76.json
Updated: /content/drive/MyDrive/dentex_test/label/test_32.json
Updated: /content/drive/MyDrive/dentex_test/label/test_136.json
Updated: /content/drive/MyDrive/dentex_test/la

In [ ]:
clean_all_jsons(LABEL_DIR, IMAGE_DIR)

Cleaned /content/drive/MyDrive/dentex_test/label/test_28.json: removed 1 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_151.json: removed 1 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_32.json: removed 1 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_198.json: removed 2 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_153.json: removed 1 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_78.json: removed 29 shapes
Deleted JSON: /content/drive/MyDrive/dentex_test/label/test_78.json
Deleted Image: /content/drive/MyDrive/dentex_test/input/test_78.png
Cleaned /content/drive/MyDrive/dentex_test/label/test_60.json: removed 2 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_201.json: removed 3 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_91.json: removed 2 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_110.json: removed 2 shapes
Cleaned /content/drive/MyDrive/dentex_test/label/test_73.j

In [ ]:
fix_impacted_label_ids(LABEL_DIR)

✅ Fixed: test_11.json
✅ Fixed: test_28.json
✅ Fixed: test_80.json
✅ Fixed: test_172.json
✅ Fixed: test_84.json
✅ Fixed: test_186.json
✅ Fixed: test_42.json
✅ Fixed: test_105.json
✅ Fixed: test_38.json
✅ Fixed: test_15.json
✅ Fixed: test_201.json
✅ Fixed: test_12.json
✅ Fixed: test_213.json
✅ Fixed: test_119.json
✅ Fixed: test_69.json
✅ Fixed: test_143.json
✅ Fixed: test_217.json
✅ Fixed: test_244.json
✅ Fixed: test_96.json
✅ Fixed: test_24.json
✅ Fixed: test_180.json
✅ Fixed: test_118.json
✅ Fixed: test_99.json
✅ Fixed: test_92.json
✅ Fixed: test_117.json
✅ Fixed: test_166.json
✅ Fixed: test_111.json
✅ Fixed: test_131.json
✅ Fixed: test_36.json
✅ Fixed: test_133.json
✅ Fixed: test_232.json
✅ Fixed: test_97.json
✅ Fixed: test_46.json
✅ Fixed: test_135.json
✅ Fixed: test_49.json
✅ Fixed: test_243.json
✅ Fixed: test_174.json
✅ Fixed: test_237.json
✅ Fixed: test_140.json
✅ Fixed: test_191.json
✅ Fixed: test_54.json
✅ Fixed: test_57.json
✅ Fixed: test_193.json
✅ Fixed: test_21.json
✅ Fixed:

In [ ]:
labelme_to_coco(LABEL_DIR, IMAGE_DIR, OUTPUT_PATH)

100%|██████████| 244/244 [08:31<00:00,  2.10s/it]


✅ COCO-style annotation saved to: /content/drive/MyDrive/dentex_test/Test_coco.json


In [ ]:
import json

# Paths
val_path = "validation_triple.json"
template_path = "abnormal-teeth-detection.json"
output_path = "validation_triple_converted.json"

# Load files
with open(val_path, "r") as f:
    val_data = json.load(f)

with open(template_path, "r") as f:
    template_data = json.load(f)

# Prepare new result
converted = {
    "name": template_data.get("name", "Regions of interest"),
    "type": template_data.get("type", "Multiple 2D bounding boxes"),
    "boxes": [],
    "version": template_data.get("version", {"major": 1, "minor": 0}),
}

# The template has "boxes" → list of dicts with "name", "corners", "probability"
# The validation_triple.json likely has bbox + category_id_1, category_id_2, category_id_3
for item in val_data:
    # Recreate the "name" string
    category_str = f"{item['category_id_1']} - {item['category_id_2']} - {item['category_id_3']}"

    x1, y1, x2, y2 = item["bbox"]  # assuming in [x1, y1, x2, y2] format
    image_id = int(item["image_id"])
    corners = [
        [x1, y1, image_id],
        [x1, y2, image_id],
        [x2, y1, image_id],
        [x2, y2, image_id],
    ]

    box_meta = {
        "name": category_str,
        "corners": corners,
        "probability": item.get("score", 1.0),
    }
    converted["boxes"].append(box_meta)

# Save result
with open(output_path, "w") as f:
    json.dump(converted, f, indent=2)

print(f"Saved converted file to {output_path}")


In [3]:
import json

# Paths
val_path = "/content/drive/MyDrive/dentex_val/validation_triple.json"
template_path = "/content/drive/MyDrive/results/abnormal-teeth-detection.json"
output_path = "/content/drive/MyDrive/dentex_val/GT.json"


# Load files
with open(val_path, "r") as f:
    val_data = json.load(f)

with open(template_path, "r") as f:
    template_data = json.load(f)

# Prepare new result matching the template structure
converted = {
    "name": template_data.get("name", "Regions of interest"),
    "type": template_data.get("type", "Multiple 2D bounding boxes"),
    "boxes": [],
    "version": template_data.get("version", {"major": 1, "minor": 0}),
}

def xywh_to_xyxy(b):
    x, y, w, h = b
    return [x, y, x + w, y + h]

# Iterate annotations (not the top-level dict)
for ann in val_data.get("annotations", []):
    # Build the "name" like "<id1> - <id2> - <id3>"
    category_str = f"{ann['category_id_1']} - {ann['category_id_2']} - {ann['category_id_3']}"

    # Convert bbox to [x1, y1, x2, y2]
    bbox = ann["bbox"]
    if len(bbox) != 4:
        raise ValueError(f"Unexpected bbox length {len(bbox)} for annotation id {ann.get('id')}")
    # Heuristic: if it's xywh (as in your file), convert to xyxy
    x1, y1, x2, y2 = xywh_to_xyxy(bbox)

    image_id = int(ann["image_id"])
    corners = [
        [x1, y1, image_id],
        [x1, y2, image_id],
        [x2, y1, image_id],
        [x2, y2, image_id],
    ]

    converted["boxes"].append({
        "name": category_str,
        "corners": corners,
        "probability": ann.get("score", 1.0),  # use 1.0 if no score available
    })

# Save result
with open(output_path, "w") as f:
    json.dump(converted, f, indent=2)

print(f"Saved converted file to {output_path} with {len(converted['boxes'])} boxes.")



Saved converted file to /content/drive/MyDrive/dentex_val/GT.json with 182 boxes.
